# Yeast9 TCA Cycle Hands-On Analysis

This notebook explores how the tricarboxylic acid (TCA) cycle in *Saccharomyces cerevisiae* responds to changing environmental conditions using the Yeast9 genome-scale metabolic model and COBRApy. It builds on the succinate-focused workflow by tracking key TCA reactions, biomass production, exchange fluxes, redox cofactor balance, and shadow prices across oxygen, pH, and temperature scenarios.


## 0. Prerequisites

Make sure your Python environment includes the core modeling stack:

```bash
pip install cobra python-dotenv pandas seaborn matplotlib networkx
```

The repository expects a `.env` file in the project root (it can be empty). It is used by the helper in `code/io.py` to locate `model/yeast-GEM.xml`.


In [ ]:
# Optional: install dependencies inside this environment
# %pip install cobra python-dotenv pandas seaborn matplotlib networkx


## 1. Imports and plotting defaults


In [ ]:
import itertools
import math
import warnings
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from cobra import Model
from cobra.flux_analysis import pfba
from code.io import read_yeast_model

sns.set_theme(style="whitegrid")
warnings.filterwarnings(
    "ignore",
    message="Solver status is 'infeasible'.",
    category=UserWarning,
    module="cobra.util.solver",
)
plt.rcParams["figure.dpi"] = 120


## 2. Load the Yeast9 model


In [ ]:
analysis_model: Model = read_yeast_model()
print(f"Loaded model with {len(analysis_model.reactions)} reactions and {len(analysis_model.metabolites)} metabolites.")


## 3. Configure media and environmental controls

We start from the Kennedy synthetic complete medium used throughout the succinate notebook and then expose helper functions to alter oxygen availability, extracellular proton exchange (as a proxy for pH), and temperature-dependent maintenance.


In [ ]:
CONSTRAINED_UPTAKE = (
    "r_1604", "r_1639", "r_1873", "r_1879", "r_1880", "r_1881", "r_1671",
    "r_1883", "r_1757", "r_1891", "r_1889", "r_1810", "r_1993", "r_1893",
    "r_1897", "r_1947", "r_1899", "r_1900", "r_1902", "r_1967", "r_1903",
    "r_1548", "r_1904", "r_2028", "r_2038", "r_1906", "r_2067", "r_1911",
    "r_1912", "r_1913", "r_2090", "r_1914", "r_2106",
)
UNCONSTRAINED_UPTAKE = (
    "r_1672", "r_1654", "r_1992", "r_2005", "r_2060", "r_1861", "r_1832",
    "r_2100", "r_4593", "r_4595", "r_4596", "r_4597", "r_2049", "r_4594",
    "r_4600", "r_2020",
)
GLUCOSE_EXCHANGE_ID = "r_1714"
OXYGEN_EXCHANGE_ID = "r_1992"
PROTON_EXCHANGE_ID = "r_1832"
AMMONIUM_EXCHANGE_ID = "r_1672"
BIOMASS_REACTION_ID = "r_4041"
NGAM_REACTION_ID = "r_4046"
SUCCINATE_EXCHANGE_ID = "r_2056"
ETHANOL_EXCHANGE_ID = "r_1761"
GLYCEROL_EXCHANGE_ID = "r_1808"

TCA_REACTIONS: Dict[str, Tuple[str, str]] = {
    "citrate_synthase": ("r_0300", "Citrate synthase"),
    "aconitase": ("r_0280", "Aconitase"),
    "isocitrate_dehydrogenase_NAD": ("r_0658", "Isocitrate dehydrogenase (NAD+)"),
    "akg_dehydrogenase_E1": ("r_0832", "α-ketoglutarate dehydrogenase (E1 component)"),
    "akg_dehydrogenase_E2": ("r_0831", "α-ketoglutarate dehydrogenase (E2 component)"),
    "succinyl_CoA_synthetase": ("r_1022", "Succinyl-CoA synthetase"),
    "succinate_dehydrogenase": ("r_1021", "Succinate dehydrogenase"),
    "fumarase": ("r_0451", "Fumarase"),
    "malate_dehydrogenase": ("r_0713", "Malate dehydrogenase"),
}

EXCHANGE_REACTIONS = {
    "glucose": GLUCOSE_EXCHANGE_ID,
    "oxygen": OXYGEN_EXCHANGE_ID,
    "ethanol": ETHANOL_EXCHANGE_ID,
    "glycerol": GLYCEROL_EXCHANGE_ID,
    "succinate": SUCCINATE_EXCHANGE_ID,
}

TEMPERATURE_NGAM = {
    "20C": 0.6,
    "30C": 0.7,
    "37C": 0.8,
}

NADH_MITO_ID = "s_1205"
FADH2_MITO_ID = "s_0690"

DEFAULT_AMMONIUM_LIMIT = -10.0
DEFAULT_GLUCOSE_LIMIT = -20.0


def apply_kennedy_medium(model: Model, glucose_limit: float = DEFAULT_GLUCOSE_LIMIT) -> None:
    for rxn in model.exchanges:
        rxn.lower_bound = 0.0
        rxn.upper_bound = 1000.0
    for rxn_id in CONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -0.5
    model.reactions.get_by_id(GLUCOSE_EXCHANGE_ID).lower_bound = glucose_limit
    for rxn_id in UNCONSTRAINED_UPTAKE:
        model.reactions.get_by_id(rxn_id).lower_bound = -1000.0


def configure_environment(
    model: Model,
    oxygen_limit: float,
    proton_limit: float,
    ammonium_limit: float,
    temperature_tag: str,
) -> None:
    model.reactions.get_by_id(OXYGEN_EXCHANGE_ID).lower_bound = oxygen_limit
    model.reactions.get_by_id(PROTON_EXCHANGE_ID).lower_bound = proton_limit
    model.reactions.get_by_id(AMMONIUM_EXCHANGE_ID).lower_bound = ammonium_limit
    ngam_limit = TEMPERATURE_NGAM.get(temperature_tag)
    if ngam_limit is None:
        raise ValueError(f"Unknown temperature tag: {temperature_tag}")
    ngam_reaction = model.reactions.get_by_id(NGAM_REACTION_ID)
    ngam_reaction.lower_bound = ngam_limit
    ngam_reaction.upper_bound = ngam_limit


### Helper functions for TCA metrics, redox balance, and shadow prices


In [ ]:
def compute_tca_redox_balance(model: Model, fluxes: pd.Series) -> Dict[str, float]:
    totals = {"net_NADH_m": 0.0, "net_FADH2_m": 0.0}
    nad_met = model.metabolites.get_by_id(NADH_MITO_ID)
    fad_met = model.metabolites.get_by_id(FADH2_MITO_ID)
    for rxn_key, (rxn_id, _) in TCA_REACTIONS.items():
        if rxn_id not in fluxes.index:
            continue
        rxn = model.reactions.get_by_id(rxn_id)
        flux = float(fluxes[rxn_id])
        if math.isclose(flux, 0.0, abs_tol=1e-9):
            continue
        if nad_met in rxn.metabolites:
            totals["net_NADH_m"] += flux * rxn.metabolites[nad_met]
        if fad_met in rxn.metabolites:
            totals["net_FADH2_m"] += flux * rxn.metabolites[fad_met]
    return totals


def summarize_shadow_prices(solution, model: Model, top_n: int = 10) -> pd.DataFrame:
    shadow = solution.shadow_prices
    table = pd.DataFrame({"met_id": shadow.index, "shadow_price": shadow.values})
    table["abs_shadow"] = table["shadow_price"].abs()
    table = table.sort_values("abs_shadow", ascending=False).head(top_n).copy()
    table["name"] = table["met_id"].map(lambda mid: model.metabolites.get_by_id(mid).name)
    table["compartment"] = table["met_id"].map(lambda mid: model.metabolites.get_by_id(mid).compartment)
    return table[["met_id", "name", "compartment", "shadow_price", "abs_shadow"]]


def evaluate_environment(
    model: Model,
    glucose_limit: float,
    oxygen_limit: float,
    proton_limit: float,
    ammonium_limit: float,
    temperature_tag: str,
):
    with model:
        apply_kennedy_medium(model, glucose_limit=glucose_limit)
        configure_environment(
            model,
            oxygen_limit=oxygen_limit,
            proton_limit=proton_limit,
            ammonium_limit=ammonium_limit,
            temperature_tag=temperature_tag,
        )
        model.objective = model.reactions.get_by_id(BIOMASS_REACTION_ID)
        solution = model.optimize()
        record = {
            "status": solution.status,
            "glucose_limit": glucose_limit,
            "oxygen_limit": oxygen_limit,
            "proton_limit": proton_limit,
            "ammonium_limit": ammonium_limit,
            "temperature_tag": temperature_tag,
            "biomass_flux": float("nan"),
        }
        for label in TCA_REACTIONS:
            record[f"flux_{label}"] = float("nan")
        for label in EXCHANGE_REACTIONS:
            record[f"exchange_{label}"] = float("nan")
        record["net_NADH_m"] = float("nan")
        record["net_FADH2_m"] = float("nan")
        if solution.status == "optimal":
            fluxes = solution.fluxes
            record["biomass_flux"] = float(solution.objective_value)
            for label, (rxn_id, _) in TCA_REACTIONS.items():
                record[f"flux_{label}"] = float(fluxes.get(rxn_id, np.nan))
            for label, rxn_id in EXCHANGE_REACTIONS.items():
                record[f"exchange_{label}"] = float(fluxes.get(rxn_id, np.nan))
            redox_totals = compute_tca_redox_balance(model, fluxes)
            record.update(redox_totals)
        return record, solution


## 4. Sweep environmental conditions

The following grid explores aerobic to anaerobic oxygen limits, proton uptake limits that mimic pH stress, two glucose feed rates, and three temperature settings. Feel free to tailor the parameter ranges before re-running the cell.


In [ ]:
oxygen_grid = (-15.0, -5.0, 0.0)
proton_grid = (-10.0, -5.0, -1.0)
glucose_grid = (-10.0, -20.0)
temperature_grid = ("20C", "30C", "37C")

records = []
solution_cache = {}
scenario_id = 0
for glucose_limit, oxygen_limit, proton_limit, temperature_tag in itertools.product(
    glucose_grid, oxygen_grid, proton_grid, temperature_grid
):
    record, solution = evaluate_environment(
        analysis_model,
        glucose_limit=glucose_limit,
        oxygen_limit=oxygen_limit,
        proton_limit=proton_limit,
        ammonium_limit=DEFAULT_AMMONIUM_LIMIT,
        temperature_tag=temperature_tag,
    )
    record["scenario_id"] = scenario_id
    records.append(record)
    solution_cache[scenario_id] = solution
    scenario_id += 1

environment_results = pd.DataFrame(records)
print(f"Simulated {len(environment_results)} environmental scenarios.")
environment_results.head()


### Inspect feasible solutions

Filter to scenarios where the LP solved successfully and rank them by biomass production.


In [ ]:
optimal_results = (
    environment_results.query("status == 'optimal'")
    .sort_values(["biomass_flux", "exchange_succinate"], ascending=[False, False])
    .reset_index(drop=True)
)
optimal_results.head()


## 5. Visualise TCA reaction behaviour

Each helper below focuses on one reaction at a time. The first produces an oxygen vs. proton heatmap for a given temperature, while the second highlights the biomass trade-off.


In [ ]:
def plot_tca_heatmap(results: pd.DataFrame, reaction_label: str, temperature_tag: str) -> None:
    column = f"flux_{reaction_label}"
    subset = results.query("status == 'optimal' and temperature_tag == @temperature_tag")
    if subset.empty:
        raise ValueError(f"No feasible points for temperature {temperature_tag}")
    pivot = subset.pivot_table(
        index="oxygen_limit",
        columns="proton_limit",
        values=column,
        aggfunc="mean",
    )
    fig, ax = plt.subplots(figsize=(5, 4))
    sns.heatmap(pivot, annot=True, fmt=".2f", cmap="mako", ax=ax)
    ax.set_title(f"{TCA_REACTIONS[reaction_label][1]} flux at {temperature_tag}")
    ax.set_xlabel("Proton lower bound (mmol/gDW/h)")
    ax.set_ylabel("Oxygen lower bound (mmol/gDW/h)")
    plt.show()


def plot_flux_tradeoff(results: pd.DataFrame, reaction_label: str) -> None:
    column = f"flux_{reaction_label}"
    subset = results.query("status == 'optimal'")
    fig, ax = plt.subplots(figsize=(5, 4))
    sns.scatterplot(
        data=subset,
        x="biomass_flux",
        y=column,
        hue="temperature_tag",
        style="glucose_limit",
        palette="viridis",
        ax=ax,
    )
    ax.set_title(f"Growth vs. {TCA_REACTIONS[reaction_label][1]} flux")
    ax.set_xlabel("Biomass flux (1/h)")
    ax.set_ylabel("Flux (mmol/gDW/h)")
    plt.show()


#### Example: citrate synthase flux landscapes


In [ ]:
plot_tca_heatmap(optimal_results, "citrate_synthase", "30C")
plot_flux_tradeoff(optimal_results, "citrate_synthase")


## 6. Redox balance and exchange readouts

Summaries of NADH/FADH₂ generation within the TCA cycle and the key exchange fluxes provide context on respiratory activity.


In [ ]:
columns_to_show = [
    "scenario_id",
    "temperature_tag",
    "glucose_limit",
    "oxygen_limit",
    "proton_limit",
    "biomass_flux",
    "exchange_oxygen",
    "exchange_ethanol",
    "exchange_glycerol",
    "exchange_succinate",
    "net_NADH_m",
    "net_FADH2_m",
]
optimal_results[columns_to_show].head()


## 7. Drill into a scenario

Pick the best-performing scenario (or another row from `optimal_results`) to inspect its full flux distribution, identify limiting metabolites via shadow prices, and visualise the TCA cycle neighbourhood.


In [ ]:
if optimal_results.empty:
    raise RuntimeError("No feasible scenarios available for inspection.")

selected_row = optimal_results.iloc[0]
selected_solution = solution_cache[selected_row["scenario_id"]]
selected_fluxes = selected_solution.fluxes
selected_pfba = pfba(analysis_model)
print("Selected scenario:
", selected_row)


### Shadow prices of limiting metabolites


In [ ]:
shadow_summary = summarize_shadow_prices(selected_solution, analysis_model, top_n=12)
shadow_summary


### Visualising the TCA cycle neighbourhood

The NetworkX helper draws a reaction-centric graph of the mitochondrial TCA steps. Node colours encode flux magnitude from the selected scenario.


In [ ]:
def plot_tca_cycle_network(model: Model, fluxes: pd.Series, max_reactions: int = None) -> None:
    reaction_items = list(TCA_REACTIONS.items())
    if max_reactions is not None:
        reaction_items = reaction_items[:max_reactions]
    graph = nx.DiGraph()
    for key, (rxn_id, label) in reaction_items:
        rxn = model.reactions.get_by_id(rxn_id)
        flux_value = float(fluxes.get(rxn_id, 0.0))
        graph.add_node(rxn_id, kind="reaction", label=label, flux=flux_value)
        for met, coeff in rxn.metabolites.items():
            graph.add_node(met.id, kind="metabolite", label=f"{met.name} [{met.compartment}]")
            if coeff < 0:
                graph.add_edge(met.id, rxn_id, weight=abs(coeff))
            else:
                graph.add_edge(rxn_id, met.id, weight=abs(coeff))
    pos = nx.spring_layout(graph, seed=42)
    reaction_nodes = [n for n, d in graph.nodes(data=True) if d.get("kind") == "reaction"]
    metabolite_nodes = [n for n, d in graph.nodes(data=True) if d.get("kind") == "metabolite"]
    flux_values = [abs(graph.nodes[n].get("flux", 0.0)) for n in reaction_nodes]
    max_flux = max(flux_values) if flux_values else 0.0
    if max_flux == 0:
        colors = [plt.cm.viridis(0.0) for _ in reaction_nodes]
    else:
        colors = [plt.cm.viridis(val / max_flux) for val in flux_values]
    fig, ax = plt.subplots(figsize=(6, 4))
    nx.draw_networkx_nodes(graph, pos, nodelist=reaction_nodes, node_shape="s", node_color=colors, node_size=400, ax=ax)
    nx.draw_networkx_nodes(graph, pos, nodelist=metabolite_nodes, node_shape="o", node_color="#fdd49e", node_size=350, ax=ax)
    nx.draw_networkx_edges(graph, pos, arrowstyle="-|>", arrowsize=12, edge_color="#555555", width=1.2, ax=ax)
    labels = {node: data["label"] for node, data in graph.nodes(data=True)}
    nx.draw_networkx_labels(graph, pos, labels=labels, font_size=7, ax=ax)
    ax.set_title("TCA cycle flux map")
    ax.axis("off")
    if max_flux > 0:
        norm = plt.Normalize(vmin=0, vmax=max_flux)
        sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
        sm.set_array([])
        fig.colorbar(sm, ax=ax, fraction=0.04, pad=0.04, label="|Flux| (mmol/gDW/h)")
    plt.show()

plot_tca_cycle_network(analysis_model, selected_fluxes)


### Compare FBA and parsimonious FBA (pFBA)

For completeness, the pFBA solution provides a flux distribution with minimal total flux while preserving the same objective. This helps confirm whether key TCA fluxes are well-defined across alternative optima.


In [ ]:
pfba_fluxes = selected_pfba.fluxes
comparison_columns = []
for label, (rxn_id, nice_name) in TCA_REACTIONS.items():
    comparison_columns.append({
        "reaction_id": rxn_id,
        "name": nice_name,
        "fba_flux": float(selected_fluxes.get(rxn_id, np.nan)),
        "pfba_flux": float(pfba_fluxes.get(rxn_id, np.nan)),
    })
pd.DataFrame(comparison_columns)


## 8. Next steps

- **Adjust the grids** to reflect experimental feed rates or oxygen transfer rates of interest.
- **Incorporate omics constraints** by setting upper bounds on reactions associated with lowly expressed genes.
- **Investigate additional metabolites** by expanding `EXCHANGE_REACTIONS` or probing shadow prices for amino acids, organic acids, or cofactors.
- **Export flux samples** using random sampling (e.g., ACHR) to map variability across feasible optima.
